## url haus

In [2]:
import requests
import pandas as pd
import psycopg2
from psycopg2 import sql
import io

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

txt_url = "https://openphish.com/feed.txt"
response = requests.get(txt_url)
txt_data = response.content.decode('utf-8')

urls = txt_data.splitlines()

df = pd.DataFrame(urls, columns=['url'])

print("Columnas del DataFrame:", df.columns)
print("Primeras filas del DataFrame:\n", df.head())

cur.execute("""
CREATE TABLE IF NOT EXISTS TablaURL4 (
    url TEXT PRIMARY KEY,
    maligna BOOLEAN NOT NULL,
    tipo TEXT NOT NULL
)
""")
conn.commit()


for _, row in df.iterrows():
    url = row['url']
    threat = 'phishing' 
    cur.execute("SELECT EXISTS(SELECT 1 FROM TablaURL4 WHERE url=%s)", (url,))
    exists = cur.fetchone()[0]

    if not exists:
        cur.execute(
            sql.SQL("INSERT INTO TablaURL4 (url, maligna, tipo) VALUES (%s, %s, %s)"),
            (url, True, threat)
        )

conn.commit()

cur.close()
conn.close()

print("Proceso completado.")


Columnas del DataFrame: Index(['url'], dtype='object')
Primeras filas del DataFrame:
                                        url
0     http://attcom-109838.weeblysite.com/
1  http://bellsouth-104558.weeblysite.com/
2             http://www.guanchejixie.com/
3                      http://0432hhh.com/
4                       http://ldexss.com/
Proceso completado.


In [2]:
import requests
import pandas as pd
import psycopg2
from psycopg2 import sql
import io

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

csv_url = 'https://urlhaus.abuse.ch/downloads/csv_recent/'
csv_url ="https://urlhaus.abuse.ch/downloads/csv_online/"
response = requests.get(csv_url)
csv_data = response.content.decode('utf-8')

csv_lines = csv_data.splitlines()
header_index = next(i for i, line in enumerate(csv_lines) if line.startswith('# id'))
header = csv_lines[header_index].lstrip('# ').split(',')
csv_data = '\n'.join(csv_lines[header_index+1:])

df = pd.read_csv(io.StringIO(csv_data), names=header)

print("Columnas del DataFrame:", df.columns)
print("Primeras filas del DataFrame:\n", df.head())

df.columns = [col.lower() for col in df.columns]

cur.execute("""
CREATE TABLE IF NOT EXISTS TablaURL (
    url TEXT PRIMARY KEY,
    maligna BOOLEAN NOT NULL,
    tipo TEXT NOT NULL
)
""")
conn.commit()

for _, row in df.iterrows():
    url = row['url']
    threat = row['threat'] if pd.notnull(row['threat']) else '1'

    # Comprobar si la URL ya está en la base de datos
    cur.execute("SELECT EXISTS(SELECT 1 FROM TablaURL WHERE url=%s)", (url,))
    exists = cur.fetchone()[0]

    if not exists:
        cur.execute(
            sql.SQL("INSERT INTO TablaURL (url, maligna, tipo) VALUES (%s, %s, %s)"),
            (url, True, threat)
        )

conn.commit()

cur.close()
conn.close()

print("Proceso completado.")

Columnas del DataFrame: Index(['id', 'dateadded', 'url', 'url_status', 'last_online', 'threat', 'tags',
       'urlhaus_link', 'reporter'],
      dtype='object')
Primeras filas del DataFrame:
         id            dateadded                                  url  \
0  3051014  2024-07-18 12:45:06        http://115.192.116.80:65023/i   
1  3051013  2024-07-18 12:44:19  http://112.248.184.222:34132/bin.sh   
2  3051012  2024-07-18 12:43:09   http://218.94.154.190:38589/bin.sh   
3  3051011  2024-07-18 12:43:06    http://59.92.189.250:40503/bin.sh   
4  3051010  2024-07-18 12:41:06       http://117.196.168.103:57991/i   

  url_status          last_online            threat                  tags  \
0     online  2024-07-18 12:45:06  malware_download  32-bit,elf,mips,Mozi   
1     online  2024-07-18 12:44:19  malware_download  32-bit,elf,mips,Mozi   
2     online  2024-07-18 12:43:09  malware_download  32-bit,elf,mips,Mozi   
3     online  2024-07-18 12:43:06  malware_download  32-bit,elf,mi

# kaggle

In [ ]:
import os
import kaggle
import pandas as pd
import psycopg2
from psycopg2 import sql
import zipfile
import shutil

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

def download_and_extract_kaggle_dataset(dataset_name, zip_file_name, dataset_path='./'):
    kaggle.api.dataset_download_files(dataset_name, path=dataset_path, unzip=False)
    zip_file = None
    for file in os.listdir(dataset_path):
        if file.endswith('.zip'):
            zip_file = os.path.join(dataset_path, file)
            break
    if not zip_file:
        raise FileNotFoundError("El archivo ZIP no se encuentra en el directorio especificado.")
    
    os.rename(zip_file, os.path.join(dataset_path, zip_file_name))

    with zipfile.ZipFile(os.path.join(dataset_path, zip_file_name), 'r') as zip_ref:
        zip_contents = zip_ref.namelist()
        print(f"Contenido del ZIP {zip_file_name}: {zip_contents}")
        zip_ref.extractall(dataset_path)
        return zip_contents

datasets = {
    'siddharthkumar25/malicious-and-benign-urls': 'malicious_and_benign_urls.zip',
    'sid321axn/malicious-urls-dataset': 'malicious_urls.zip'
}

csv_files = {}

for dataset, zip_file_name in datasets.items():
    csv_files[dataset] = download_and_extract_kaggle_dataset(dataset, zip_file_name)

csv_file_1 = 'urldata.csv'  
df1 = pd.read_csv(csv_file_1)

cur.execute("""
CREATE TABLE IF NOT EXISTS TablaURL (
    url TEXT PRIMARY KEY,
    maligna BOOLEAN NOT NULL,
    tipo TEXT NOT NULL
)
""")
conn.commit()

for _, row in df1.iterrows():
    url = row['url']
    maligna = row['result'] == 1
    tipo = 'malicious' if maligna else 'benign'

    cur.execute("SELECT EXISTS(SELECT 1 FROM TablaURL WHERE url=%s)", (url,))
    exists = cur.fetchone()[0]

    if not exists:
        cur.execute(
            sql.SQL("INSERT INTO TablaURL (url, maligna, tipo) VALUES (%s, %s, %s)"),
            (url, maligna, tipo)
        )

conn.commit()

shutil.move(csv_file_1, f'{csv_file_1}_backup')

csv_file_2 = 'malicious_phish.csv'  
df2 = pd.read_csv(csv_file_2)

for _, row in df2.iterrows():
    url = row['url']
    maligna = row['type'] != 'benign'
    tipo = row['type']

    cur.execute("SELECT EXISTS(SELECT 1 FROM TablaURL WHERE url=%s)", (url,))
    exists = cur.fetchone()[0]

    if not exists:
        cur.execute(
            sql.SQL("INSERT INTO TablaURL (url, maligna, tipo) VALUES (%s, %s, %s)"),
            (url, maligna, tipo)
        )

conn.commit()

cur.close()
conn.close()

print("Proceso completado.")

## Palabras sospechosas

In [ ]:
import psycopg2
from psycopg2 import sql
import requests

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

suspicious_words_url = 'https://raw.githubusercontent.com/danielmiessler/SecLists/master/Discovery/Web-Content/raft-large-words.txt'
common_passwords_url = 'https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10k-most-common.txt'

def download_and_read_text(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text.splitlines()

suspicious_words = download_and_read_text(suspicious_words_url)
common_passwords = download_and_read_text(common_passwords_url)

cur.execute("""
CREATE TABLE IF NOT EXISTS Palabras (
    id SERIAL PRIMARY KEY,
    palabra TEXT NOT NULL,
    tipo TEXT NOT NULL
)
""")
conn.commit()

def insert_words(words, tipo):
    data_to_insert = [(word, tipo) for word in words]
    cur.executemany(
        sql.SQL("INSERT INTO Palabras (palabra, tipo) VALUES (%s, %s) ON CONFLICT DO NOTHING"),
        data_to_insert
    )
    conn.commit()

insert_words(suspicious_words, 'suspicious')
insert_words(common_passwords, 'common_password')

cur.close()
conn.close()

print("Proceso completado.")


### Tablas WHOIS y Geolocalización

In [ ]:
import psycopg2

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS WHOIS (
    id SERIAL PRIMARY KEY,
    domain TEXT NOT NULL,
    creation_date DATE,
    expiration_date DATE,
    registrant_name TEXT,
    registrant_organization TEXT,
    registrant_country TEXT,
    FOREIGN KEY (domain) REFERENCES TablaURL (url)
)
""")
conn.commit()

cur.execute("""
CREATE TABLE IF NOT EXISTS Geolocalizacion (
    id SERIAL PRIMARY KEY,
    ip TEXT NOT NULL,
    country TEXT,
    region TEXT,
    city TEXT,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    FOREIGN KEY (ip) REFERENCES TablaURL (url)
)
""")
conn.commit()

cur.close()
conn.close()

print("Tablas WHOIS y Geolocalizacion creadas exitosamente.")

### tdl

In [ ]:
import requests
import psycopg2
from psycopg2 import sql
from bs4 import BeautifulSoup

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS TLDs (
    id SERIAL PRIMARY KEY,
    domain TEXT NOT NULL UNIQUE,
    type TEXT,
    tld_manager TEXT
)
""")
conn.commit()

def get_tlds_from_iana():
    url = 'https://www.iana.org/domains/root/db'
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    tld_list = []

    for row in soup.select('table tbody tr'):
        columns = row.find_all('td')
        if len(columns) >= 3:
            domain = columns[0].text.strip()
            tld_type = columns[1].text.strip()
            tld_manager = columns[2].text.strip()
            tld_list.append((domain, tld_type, tld_manager))

    return tld_list

tlds = get_tlds_from_iana()

cur.executemany(
    sql.SQL("INSERT INTO TLDs (domain, type, tld_manager) VALUES (%s, %s, %s) ON CONFLICT (domain) DO NOTHING"),
    tlds
)
conn.commit()
cur.close()
conn.close()

print("Tabla TLDs creada y poblada exitosamente.")


### Url Shorteners

In [ ]:
import requests
import psycopg2

url = 'https://raw.githubusercontent.com/Spamfighter666/Short-URL-Providers-List/master/Short-URL-Providers-list.txt'

response = requests.get(url)
short_url_providers = response.text.splitlines()

short_url_providers = [line for line in short_url_providers if line]

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS ShortURL (
        id SERIAL PRIMARY KEY,
        url_provider VARCHAR(255) NOT NULL
    )
""")
conn.commit()

insert_query = "INSERT INTO ShortURL (url_provider) VALUES (%s)"
for provider in short_url_providers:
    cur.execute(insert_query, (provider,))

conn.commit()

cur.close()
conn.close()


http://110.90.122.245:9991/Photo.lnk


In [4]:
import psycopg2
import requests
from psycopg2.extras import execute_batch

db_config = {
    'dbname': 'url',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432',
    'options': '-c client_encoding=UTF8'
}

BATCH_SIZE = 100  
OFFSET = 5000  

def check_url_online(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def main():
    try:
        
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()
        
        cur.execute("""
            CREATE TABLE IF NOT EXISTS urls (
                url TEXT PRIMARY KEY,
                maligna BOOLEAN NOT NULL,
                tipo TEXT NOT NULL
                
            )
        """)
        conn.commit()

        cur.execute("""
            SELECT url, tipo 
            FROM TablaURL2 
            WHERE maligna = True 
            OFFSET %s
        """, (OFFSET,))
        urls = cur.fetchall()
        for i in range(0, len(urls), BATCH_SIZE):
            batch = urls[i:i + BATCH_SIZE]
            valid_urls = []
            
            for url, tipo in batch:
                if check_url_online(url):
                    valid_urls.append((url, True ,tipo))

            if valid_urls:
                execute_batch(cur, """
                    INSERT INTO urls (url, maligna, tipo) VALUES (%s, %s, %s)
                    ON CONFLICT (url) DO NOTHING
                """, valid_urls)
                conn.commit()

            print(f"Lote {i // BATCH_SIZE + 1}: Se han insertado {len(valid_urls)} URLs válidas en la nueva tabla.")
        
    except psycopg2.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
    finally:
        if conn:
            cur.close()
            conn.close()

if __name__ == "__main__":
    main()


Lote 1: Se han insertado 75 URLs válidas en la nueva tabla.
Lote 2: Se han insertado 88 URLs válidas en la nueva tabla.
Lote 3: Se han insertado 78 URLs válidas en la nueva tabla.
Lote 4: Se han insertado 82 URLs válidas en la nueva tabla.
Lote 5: Se han insertado 73 URLs válidas en la nueva tabla.
Lote 6: Se han insertado 100 URLs válidas en la nueva tabla.
Lote 7: Se han insertado 95 URLs válidas en la nueva tabla.
Lote 8: Se han insertado 95 URLs válidas en la nueva tabla.
Lote 9: Se han insertado 100 URLs válidas en la nueva tabla.
Lote 10: Se han insertado 91 URLs válidas en la nueva tabla.
Lote 11: Se han insertado 100 URLs válidas en la nueva tabla.
Lote 12: Se han insertado 89 URLs válidas en la nueva tabla.
Lote 13: Se han insertado 71 URLs válidas en la nueva tabla.
Lote 14: Se han insertado 73 URLs válidas en la nueva tabla.
Lote 15: Se han insertado 84 URLs válidas en la nueva tabla.
Lote 16: Se han insertado 95 URLs válidas en la nueva tabla.
Lote 17: Se han insertado 100 

In [ ]:
import psycopg2
import requests
from psycopg2.extras import execute_batch

db_config = {
    'dbname': 'url',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432',
    'options': '-c client_encoding=UTF8'
}

BATCH_SIZE = 100 
OFFSET = 12400

def check_url_online(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def main():
    try:
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()
        
        # Crear la nueva tabla si no existe
        cur.execute("""
            CREATE TABLE IF NOT EXISTS urls_online (
                url TEXT PRIMARY KEY,
                maligna BOOLEAN NOT NULL,
                tipo TEXT NOT NULL
                
            )
        """)
        conn.commit()

        cur.execute("""
            SELECT url, tipo 
            FROM TablaURL 
            WHERE maligna = True 
            OFFSET %s
        """, (OFFSET,))
        urls = cur.fetchall()
        for i in range(0, len(urls), BATCH_SIZE):
            batch = urls[i:i + BATCH_SIZE]
            valid_urls = []
            
            for url, tipo in batch:
                if check_url_online(url):
                    valid_urls.append((url, True ,tipo))

            # Insertar las URLs válidas en la nueva tabla
                execute_batch(cur, """
                    INSERT INTO urls_online (url, maligna, tipo) VALUES (%s, %s, %s)
                    ON CONFLICT (url) DO NOTHING
                """, valid_urls)
                conn.commit()

            print(f"Lote {i // BATCH_SIZE + 1}: Se han insertado {len(valid_urls)} URLs válidas en la nueva tabla.")
        
    except psycopg2.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
    finally:
        if conn:
            cur.close()
            conn.close()

if __name__ == "__main__":
    main()


Lote 1: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 2: Se han insertado 1 URLs válidas en la nueva tabla.
Lote 3: Se han insertado 1 URLs válidas en la nueva tabla.
Lote 4: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 5: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 6: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 7: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 8: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 9: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 10: Se han insertado 1 URLs válidas en la nueva tabla.
Lote 11: Se han insertado 3 URLs válidas en la nueva tabla.
Lote 12: Se han insertado 0 URLs válidas en la nueva tabla.
Lote 13: Se han insertado 5 URLs válidas en la nueva tabla.
Lote 14: Se han insertado 11 URLs válidas en la nueva tabla.
Lote 15: Se han insertado 5 URLs válidas en la nueva tabla.
Lote 16: Se han insertado 2 URLs válidas en la nueva tabla.
Lote 17: Se han insertado 6 URLs válidas en la n

In [ ]:
import psycopg2
from psycopg2 import sql
import requests

conn = psycopg2.connect(
    dbname="url",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

suspicious_words_url = 'https://raw.githubusercontent.com/danielmiessler/SecLists/master/Discovery/Web-Content/raft-large-words.txt'
common_passwords_url = 'https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10k-most-common.txt'

def download_and_read_text(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text.splitlines()

suspicious_words = download_and_read_text(suspicious_words_url)
common_passwords = download_and_read_text(common_passwords_url)

cur.execute("""
CREATE TABLE IF NOT EXISTS Palabras (
    id SERIAL PRIMARY KEY,
    palabra TEXT NOT NULL,
    tipo TEXT NOT NULL
)
""")
conn.commit()

def insert_words(words, tipo):
    data_to_insert = [(word, tipo) for word in words]
    cur.executemany(
        sql.SQL("INSERT INTO Palabras (palabra, tipo) VALUES (%s, %s) ON CONFLICT DO NOTHING"),
        data_to_insert
    )
    conn.commit()

insert_words(suspicious_words, 'suspicious')
insert_words(common_passwords, 'common_password')

cur.close()
conn.close()

print("Proceso completado.")
